In [1]:
import numpy as np
import pandas as pd
from joblib import load
from processed_feature_mapping import mapping
from train_model import data_cleaning
from model_monitoring import ModelMonitoring

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [2]:
# 1a. Output the required cleaned data for train dataset
# Required processed data is already done in train_model.py
# Although prediction on employee_train was already done previously,
# need to do again because the dataset was jumbled up in train_model.py
train_df = pd.read_csv('../data/raw_split_data/employee_train.csv')
train_df = data_cleaning(train_df)
X_train = train_df.drop(columns=['Attrition'])

column_transformer = load('./preprocessor/column_transformer.pkl')
label_encoder = load('./preprocessor/label_encoder.pkl')
RF_clf = load('./model/RF_clf.joblib')

y_train = train_df['Attrition']
X_train_processed = column_transformer.transform(X_train)
y_train_pred = RF_clf.predict(X_train_processed)
y_train_pred_inverse = label_encoder.inverse_transform(y_train_pred)
train_df['prediction'] = y_train_pred_inverse
train_df.rename(columns={'Attrition' : 'target'}, inplace=True)
train_df.to_csv('../data/cleaned_employee_train.csv', index=False)

# 1b. Output the required cleaned/processed data for test dataset
test_df = pd.read_csv("../data/raw_split_data/employee_test.csv")
test_df = data_cleaning(test_df)
X_test = test_df.drop(columns=['Attrition'])
y_test = test_df['Attrition']
X_test_processed = column_transformer.transform(X_test)
X_test_processed = pd.DataFrame.from_records(X_test_processed)
X_test_processed = mapping(X_test_processed, column_transformer)
X_test_processed.to_csv('../data/X_test_processed.csv', index=False)
y_test_pred = RF_clf.predict(X_test_processed)
y_test_pred_inverse = label_encoder.inverse_transform(y_test_pred)
y_test_pred_prob = RF_clf.predict_proba(X_test_processed)[:1]
test_df['prediction'] = y_test_pred_inverse
test_df.rename(columns={'Attrition' : 'target'}, inplace=True)
test_df.to_csv('../data/cleaned_employee_test.csv', index=False)

In [3]:
# With the required datasets out, firstly, we create the instance of the ModelMonitoring object 
full_train_df = pd.read_csv('../data/cleaned_employee_train.csv')
full_test_df = pd.read_csv('../data/cleaned_employee_test.csv')
train_df = full_train_df.drop(columns=['target', 'prediction'])
test_df = full_test_df.drop(columns=['target', 'prediction'])

processed_train_df = pd.read_csv('../data/X_train_processed.csv')
processed_test_df = pd.read_csv('../data/X_test_processed.csv')
model_monitoring = ModelMonitoring(train_df)

we first have to ensure that the incoming data has no issues
for example, that the column names of the incoming data matches the training data, 
there is no difference in the data type for all columns, 
there is no data with '?' or '-' within the dataset.

In [4]:
# so we first have to run the data_check method
# this method will fix any column names that has spaces and change them to '_'
# it then replaces any bad data such as '?' or '-' and replaces the with NaN
# it then checks whether the schema of the incoming data matches the training data
# then it checks for the data types for each column of the incoming data
# lastly it also checks that the processed incoming data is the same as the processed training data
# this is to ensure the data quality of the incoming data and that the
# processing pipeline has no issues
model_monitoring.data_check(train_df, test_df, processed_train_df, processed_test_df)

train and test dataset have the same features
The data types for all columns from both datasets are the same
train and test dataset have the same processed features


In [ ]:
# after ensuring that there is no issue with the incoming data, can run the evidently reports
# the report can be obtained in 2 formats: html or json
# html can be used for visualizing the distribution of the data
# json can be used for raising alerts as the user can access the p-value of the stat test
# of each column and should many columns drifted, it can raise an alert to let the 
# data scientist know that the dataset has drifted and retraining might be in order
# this function will always return a dictionary should the user want to access the data within this ipynb
# you can view either in html or json format
model_monitoring.feature_drift_report(train_df, test_df, 'html')
# model_monitoring.feature_drift_report(train_df, test_df, 'json')


In [6]:
# on top of the feature report, we can also monitor the prediction column to see 
# how the prediction has shifted
# model_monitoring.prediction_drift_report(full_train_df, full_test_df, 'html')
model_monitoring.prediction_drift_report(full_train_df, full_test_df, 'json')

{'metrics': [{'metric': 'ColumnDriftMetric',
   'result': {'column_name': 'prediction',
    'column_type': 'cat',
    'stattest_name': 'Z-test p_value',
    'stattest_threshold': 0.05,
    'drift_score': 0.018051453253177918,
    'drift_detected': True,
    'current': {'small_distribution': {'x': ['No', 'Yes'], 'y': [658, 77]}},
    'reference': {'small_distribution': {'x': ['No', 'Yes'],
      'y': [628, 107]}}}},
  {'metric': 'ColumnCorrelationsMetric',
   'result': {'column_name': 'prediction',
    'current': {'cramer_v': {'column_name': 'prediction',
      'kind': 'cramer_v',
      'values': {'x': ['Department',
        'StockOptionLevel',
        'Education',
        'RelationshipSatisfaction',
        'JobInvolvement',
        'WorkLifeBalance',
        'JobSatisfaction',
        'EducationField',
        'Gender',
        'OverTime',
        'BusinessTravel',
        'JobLevel',
        'PerformanceRating',
        'JobRole',
        'MaritalStatus',
        'EnvironmentSatisfac